# Modeling

## Prepare Dataset

In [1]:
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import math
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [34]:
train_path = r'./train_data.csv'

In [3]:
train_df = pd.read_csv(train_path)

In [4]:
train_df.head()

,sequence_id,sequence,experiment_type,dataset_name,reactivity_0001,reactivity_0002,reactivity_0003,reactivity_0004,reactivity_0005,reactivity_0006,...,reactivity_error_0197,reactivity_error_0198,reactivity_error_0199,reactivity_error_0200,reactivity_error_0201,reactivity_error_0202,reactivity_error_0203,reactivity_error_0204,reactivity_error_0205,reactivity_error_0206
0,0000d87cab97,GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...,2A3_MaP,DasLabBigLib_OneMil_RFAM_windows_100mers_2A3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000d87cab97,GGGAACGACUCGAGUAGAGUCGAAAAAGAUCGCCACGCACUUACGA...,DMS_MaP,DasLabBigLib_OneMil_RFAM_windows_100mers_DMS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0001ca9d21b0,GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...,2A3_MaP,DasLabBigLib_OneMil_OpenKnot_Round_2_train_2A3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001ca9d21b0,GGGAACGACUCGAGUAGAGUCGAAAAGGUGGCCGGCAGAAUCGCGA...,DMS_MaP,DasLabBigLib_OneMil_OpenKnot_Round_2_train_DMS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00021f968267,GGGAACGACUCGAGUAGAGUCGAAAACAUUGUUAAUGCCUAUAUUA...,2A3_MaP,DasLabBigLib_OneMil_Replicates_from_previous_l...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
train, val = train_test_split(train_df, test_size=0.2, random_state=283)

In [35]:
max_seq_length = 407
nucleotides = 'ACGU'

In [36]:
def str_to_seq(s):
    mapping = {nucleotide: idx for idx, nucleotide in enumerate(nucleotides)}
    return [mapping[c] for c in s]


def train_test_split(
        df: pd.DataFrame,
        test_size: float,
        random_state: int
):
    n = len(df)
    rng = np.random.default_rng(seed=random_state)
    idx = rng.permutation(n)
    split = math.floor(n * test_size)
    return df.iloc[idx[split:]], df.iloc[idx[:split]]

def extract_input_seq(df: pd.DataFrame, idx):
    input_seq = df['sequence'].iloc[idx]
    input_seq = str_to_seq(input_seq)
    input_seq = torch.LongTensor(input_seq)
    input_seq = F.one_hot(input_seq, num_classes=len(nucleotides))
    input_seq = input_seq.float()
    input_seq = F.pad(input_seq, pad=(0, 0, 0, max_seq_length - input_seq.size(0)))

    return input_seq


def extract_label_seq(df: pd.DataFrame, label_idx, idx):
    label_seq = df.iloc[idx, label_idx]
    label_seq = torch.FloatTensor(label_seq.to_numpy(dtype=float))
    label_seq = torch.nan_to_num(label_seq)
    label_seq = F.pad(label_seq, pad=(0, max_seq_length - label_seq.size(0)))

    return label_seq


class RNADataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.label_idx = [idx for idx, column in enumerate(self.df.columns) if
                          not column.startswith('reactivity_error') and column.startswith('reactivity')]
        self.err_idx = [idx for idx, column in enumerate(self.df.columns) if
                        column.startswith('reactivity_error')]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_seq = extract_input_seq(self.df, idx)
        label_seq = extract_label_seq(self.df, self.label_idx, idx)
        err_seq = extract_label_seq(self.df, self.err_idx, idx)
        return input_seq, label_seq, len(self.df['sequence'].iloc[idx]), int(self.df['experiment_type'].iloc[idx] == 'DMS_MaP')


class RNAPredictDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_seq = extract_input_seq(self.df, idx)

        return input_seq #, len(self.df['sequence'].iloc[idx])


In [8]:
batch_size = 128

In [9]:
train_loader = DataLoader(RNADataset(train), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(RNADataset(val), batch_size=batch_size, shuffle=True)

## Define and Train Model

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)


class RNAModel(nn.Module):
    def __init__(self, embed_dim, d_model, nhead, num_layers, dropout):
        super().__init__()

        self.conv = nn.Conv1d(embed_dim, d_model, kernel_size=3, padding=1)
        self.pe = PositionalEncoding(d_model=d_model, dropout=dropout)
        self.te = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True),
            num_layers=num_layers
        )
        self.linear = nn.Linear(d_model, 2)

    def forward(self, x):
        x = self.conv(x.transpose(-1, -2)).transpose(-1, -2)
        x = self.pe(x)
        x = self.te(x)
        x = self.linear(x)

        return x.squeeze()

In [12]:
model = RNAModel(
    embed_dim=len(nucleotides),
    d_model=128,
    nhead=4,
    num_layers=4,
    dropout=.01
).to(device)

criterion = nn.MSELoss()
mae = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
num_epochs = 5

In [15]:
loss_train = np.zeros(num_epochs)
loss_valid = np.zeros(num_epochs)
loss_mae_valid = np.zeros(num_epochs)

for epoch in tqdm(range(num_epochs)):
    train_size = 0
    valid_size = 0

    # train for one epoch
    model.train()
    for inputs, labels, seq_lengths, is_dmp in  train_loader:
        n_batch_train = len(inputs)
        inputs, labels, is_dmp = inputs.to(device), labels.to(device), is_dmp.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs[torch.arange(outputs.size(0)), :, is_dmp]
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_train[epoch] += loss.item() * n_batch_train
        train_size += n_batch_train
    
    # validation
    with torch.no_grad():
        model.eval()
        for inputs, labels, seq_lengths, is_dmp in val_loader:
            n_batch_valid = len(inputs)
            inputs, labels, is_dmp = inputs.to(device), labels.to(device), is_dmp.to(device)
            outputs = model(inputs)
            outputs = outputs[torch.arange(outputs.size(0)), :, is_dmp]
            loss = criterion(outputs, labels)
            loss_valid[epoch] += loss.item() * n_batch_valid
            mae_loss = mae(outputs, labels)
            loss_mae_valid[epoch] += mae_loss.item() * n_batch_valid
            valid_size += n_batch_valid
    
    loss_train[epoch] /= train_size
    loss_valid[epoch] /= valid_size
    loss_mae_valid[epoch] /= valid_size
    print("epoch {}, train loss : {}, valid loss : {}, valid mae : {}".format(epoch, loss_train[epoch], loss_valid[epoch], loss_mae_valid[epoch]))

 20%|██        | 1/5 [07:08<28:35, 428.95s/it]

epoch 0, train loss : 0.15595159970190936, valid loss : 0.12590118477275392, valid mae : 0.15653921639785728


 40%|████      | 2/5 [14:17<21:26, 428.91s/it]

epoch 1, train loss : 0.12322047734671522, valid loss : 0.11751575883075895, valid mae : 0.14972028674989696


 60%|██████    | 3/5 [21:33<14:23, 431.88s/it]

epoch 2, train loss : 0.11577148947597762, valid loss : 0.11190346149274501, valid mae : 0.14847095789397036


 80%|████████  | 4/5 [28:47<07:12, 432.63s/it]

epoch 3, train loss : 0.11125850759795934, valid loss : 0.10853968367471604, valid mae : 0.1375571650138068


100%|██████████| 5/5 [36:01<00:00, 432.28s/it]

epoch 4, train loss : 0.10879031719146978, valid loss : 0.10599314082030083, valid mae : 0.13851197617133265


In [37]:
test_df = pd.read_csv('test_sequences.csv')
seq_len = [len(seq) for seq in test_df.sequence.to_numpy()]
testloader = DataLoader(RNAPredictDataset(test_df), batch_size=128, shuffle=False)

In [82]:
torch.cuda.empty_cache()

In [39]:
for i in testloader:
    print(i.shape)
    break

torch.Size([128, 407, 4])


In [60]:
predictions = []
with torch.no_grad():
    model.eval()
    for batch in tqdm(testloader):
        batch = batch.to(device)
        outputs = model(batch)
        predictions.append(outputs.cpu().numpy())

100%|██████████| 10499/10499 [15:39<00:00, 11.17it/s]


In [80]:
# np.append too slow

modified_preds = []
i = 0
for batch_preds in tqdm(predictions):
    for j, pred in enumerate(batch_preds):
        # print(pred.shape)
        # print(pred[:seq_len[i+j]].shape)
        modified_preds.append(pred[:seq_len[i+j]]) 
    i += j + 1
print(modified_preds[0][0])
modified_preds = np.concatenate(modified_preds)
print(modified_preds[0])
print(modified_preds.shape)

100%|██████████| 10499/10499 [00:00<00:00, 19995.65it/s]


[-0.0018136  0.0070217]
[-0.0018136  0.0070217]
(269496671, 2)


In [84]:
df = pd.DataFrame({
    'id': np.arange(modified_preds.shape[0]),
    'reactivity_DMS_MaP': modified_preds[:, 1],
    'reactivity_2A3_MaP': modified_preds[:, 0]
})

In [86]:
df.to_csv('submission.csv.gz', index=False, compression='gzip')